In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer


In [ ]:

data = pd.read_excel('Belly Button Batch2_OTU and Metadata_17Nov14.xlsx', sheet_name=1)
data = data[['ID', 'Ethnicity_Quest', 'Gender', 'Age', 'Wash_Freq of BB (# times per week)', 'Current City']]
data.columns = data.columns.str.strip()  

In [ ]:

data = data.replace('Unknown', np.nan)


cat_cols = ['Ethnicity_Quest', 'Gender', 'Current City']
encoder = OneHotEncoder(sparse=False, drop='first')
encoded = encoder.fit_transform(data[cat_cols])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_cols))


c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:

data.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)


non_cat_cols = data.drop(columns=cat_cols)
for col in non_cat_cols.columns:
    if col != 'ID':
        non_cat_cols[col] = pd.to_numeric(non_cat_cols[col], errors='coerce')


In [ ]:

final_df = pd.concat([non_cat_cols, encoded_df], axis=1)


imputer = KNNImputer(n_neighbors=3)
df_impute = final_df.copy()

imputed_array = imputer.fit_transform(df_impute.drop(columns=['ID']))
imputed_df = pd.DataFrame(imputed_array, columns=df_impute.columns.drop('ID'))
imputed_df['ID'] = df_impute['ID'].values
imputed_df = imputed_df[['ID'] + [col for col in imputed_df.columns if col != 'ID']]


print(imputed_df[['ID', 'Wash_Freq of BB (# times per week)']].head())

     ID  Wash_Freq of BB (# times per week)
0  1153                            2.333333
1  1161                            2.333333
2  1169                            2.750000
3   979                            2.333333
4  1113                            4.666667


In [51]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Drop ID — this was the actual imputation input
X_used = df_impute.drop(columns=['ID'])

# Reduce dimensionality
pca = PCA(n_components=2)
X_2D = pca.fit_transform(X_used)

# Identify which rows were imputed
was_missing = df_impute['Wash_Freq of BB (# times per week)'].isna()

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(X_2D[~was_missing, 0], X_2D[~was_missing, 1], c='blue', label='Original Known', s=60)
plt.scatter(X_2D[was_missing, 0], X_2D[was_missing, 1], c='red', marker='x', label='Imputed', s=80)

plt.title('KNN Imputation (PCA projection of features used)')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values